In [1]:
# Standard imports

# get the environment
import sys
print(sys.version)
print(sys.executable)
import os
os.getcwd()

3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
C:\ProgramData\Anaconda3\python.exe


'M:\\bbutler\\PLoan'

In [2]:
# import base libraries
import pandas as pd
import numpy as np


# set options
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 100)
pd.options.display.float_format = '{:.4f}'.format

# Overview
## This model is designed to take in loan terms (contract amount, term and apr) and then
## returns the financial values of each loan: Monthly Payment, Interest Income,
## Total Costs, Profit and NPV of Profit

# Get the Data and Create a Function to Calculate Monthly Payment

In [3]:
# set the directory
loanDir = 'M:\\bbutler\\PLoan\\CRIF_210301'

os.chdir(loanDir)

# get the file
# bring in balances

fileName = 'PLoanPort2021.csv'

ploanPort = pd.read_csv(fileName,low_memory=False)
ploanPort.head()

,AppId,ContractNum,contractAmt,Term,Interest Rate
0,65398,3750024720,20000,60,9.7500
1,65413,3750024698,18000,60,9.4900
2,65423,3750024577,18000,60,8.7500
3,65435,3750024546,13000,60,11.4900
4,65441,3750024686,4500,60,11.4900


In [4]:
# change the interest rate 

ploanPort['Interest Rate'] = ploanPort['Interest Rate']/100
ploanPort.columns = ['AppId', 'ContractNum', 'ContractAmt', 'Term', 'APR']
ploanPort['AppId'] = ploanPort['AppId'].astype(str)
ploanPort['ContractNum'] = ploanPort['ContractNum'].astype(str)
ploanPort.head()

,AppId,ContractNum,ContractAmt,Term,APR
0,65398,3750024720,20000,60,0.0975
1,65413,3750024698,18000,60,0.0949
2,65423,3750024577,18000,60,0.0875
3,65435,3750024546,13000,60,0.1149
4,65441,3750024686,4500,60,0.1149


In [5]:
# formula to calculate the payment
def calcPayment(origBalance, term, apr):
    """
    Function calculates the monthly payment for a Loan given the balance, term and apr
    
    term: should be a period in months
    
    
    """
    # break formula into numerator and denominator
    numerator = (origBalance * (apr/12) * (1+(apr/12))**term)
    denominator = (((1+(apr/12))**term) - 1)
    
    monthPayment = numerator/denominator
    
    return monthPayment

In [6]:
# create payment 

ploanPort['CalcPayment'] = ploanPort.apply(lambda df: calcPayment(df.ContractAmt, df.Term, df.APR), axis = 1)
ploanPort.head()

,AppId,ContractNum,ContractAmt,Term,APR,CalcPayment
0,65398,3750024720,20000,60,0.0975,422.4849
1,65413,3750024698,18000,60,0.0949,377.9455
2,65423,3750024577,18000,60,0.0875,371.4702
3,65435,3750024546,13000,60,0.1149,285.8386
4,65441,3750024686,4500,60,0.1149,98.9441


# Set up Key Inputs & Model Parameters

## The model utilizes two data frames to perform all calculations.
## 1. Input dataframe contains the loan ID, and loan details: contract amt, term and apr
## 2. The second dataframe calculates the details of the loan for each month of the loan term and applies discounting to generate the NPV of the loan.

## The model loops through all of the loans in the 1st dataframe to generate a 2nd temporary dataframe for each loan which is rolled up and the values are transferred back to the 1st dataframe with the rest of the financial resolts.

# Step 1: Calculate Interest (This is the Loan Revenue)

In [7]:
ploanPort['Interest'] = (ploanPort['CalcPayment']*ploanPort['Term']) - ploanPort['ContractAmt']
ploanPort.head()

,AppId,ContractNum,ContractAmt,Term,APR,CalcPayment,Interest
0,65398,3750024720,20000,60,0.0975,422.4849,5349.0924
1,65413,3750024698,18000,60,0.0949,377.9455,4676.7327
2,65423,3750024577,18000,60,0.0875,371.4702,4288.2113
3,65435,3750024546,13000,60,0.1149,285.8386,4150.3182
4,65441,3750024686,4500,60,0.1149,98.9441,1436.6486


# Build out the 2nd dataframe of loan details (revenue and costs)

In [8]:
# drop AppID
df = ploanPort[['AppId']].copy()
df['AppId'] = df['AppId'].astype('str')
df.head()


,AppId
0,65398
1,65413
2,65423
3,65435
4,65441


In [9]:
ploanPort = ploanPort.drop('AppId', axis = 1)
ploanPort.head()

,ContractNum,ContractAmt,Term,APR,CalcPayment,Interest
0,3750024720,20000,60,0.0975,422.4849,5349.0924
1,3750024698,18000,60,0.0949,377.9455,4676.7327
2,3750024577,18000,60,0.0875,371.4702,4288.2113
3,3750024546,13000,60,0.1149,285.8386,4150.3182
4,3750024686,4500,60,0.1149,98.9441,1436.6486


In [10]:
numLoans = len(ploanPort)
numLoans

118

In [11]:
# key parameters
numLoans = len(ploanPort)
ln = 0

while ln < numLoans:

    ########################  MODEL INPUTS FROM THE CSV
    monthly = ploanPort['CalcPayment'][ln]
    orig = ploanPort['ContractAmt'][ln]
    term = ploanPort['Term'][ln].astype(int)
    rate = ploanPort['APR'][ln]


    ######################  COST PARAMS TO BE PROVIDED BY PRODUCT
    # inputs for loan costs
    costOfFunds = .025
    provisions = .01
    servicingCosts = 13.34
    nplRate = .0025
    originationCosts = 54.72
    earlyPayoff = .0043
    docSupport = 19.56
    discountRate = .10

    #############    BEGIN THE DATAFRAME BUILD
    # populate period for first loan
    periods = np.array(range(0,term + 1))

    # create the empty data frame
    dict = {'Period': periods}

    loanDf = pd.DataFrame(dict)

    # fill in the monthly
    loanDf['MonthlyPayment'] = monthly

    # zero out monthly payment for first month
    loanDf.loc[0, 'MonthlyPayment'] = 0

    # get the beginning balance
    loanDf['BeginBal'] = orig

    # now iterate through to generate outstanding balance for each subsequent term
    loanDf.loc[0, 'OutBal'] = loanDf.loc[0, 'BeginBal'] - loanDf.loc[0, 'MonthlyPayment']

    for i in range(1, len(loanDf)):
        loanDf.loc[i, 'BeginBal'] = loanDf.loc[i - 1, 'OutBal']
        loanDf.loc[i, 'OutBal'] = loanDf.loc[i, 'BeginBal'] * (1 + rate/12) - loanDf.loc[i, 'MonthlyPayment']

    # then drop row zero will only need to calculate for terms 1 to n (end)
    loanDf = loanDf.drop(0)

    # create interest income
    loanDf['IntIncome'] = (loanDf['BeginBal']*(1+rate/12))-loanDf['BeginBal']


    # create all of the cost columns from the inputs
    # append them all to a list
    loanDf['CostOfFunds'] = loanDf['OutBal'] * costOfFunds/12
    loanDf['Provisions'] = loanDf['OutBal'] * provisions/12
    loanDf['ServicingCosts'] = loanDf['Period'].apply(lambda x: servicingCosts if x < term else 0)
    loanDf['OriginationCosts'] = originationCosts/term
    loanDf['DocSupport'] = docSupport/term
    loanDf['EarlyPayment'] = loanDf['OutBal'] * earlyPayoff/12


    # select the columns to add up for costs
    costCols = ['CostOfFunds', 'Provisions', 'ServicingCosts',
             'OriginationCosts', 'DocSupport', 'EarlyPayment']

    # make a total cost by summing up the cost columns
    loanDf['TotalCost'] = loanDf.loc[:,costCols].sum(axis=1)

    # calculate undiscounted profit/loss
    loanDf['Profit'] = loanDf['IntIncome'] - loanDf['TotalCost']

    # calculate NPV for the loan at each term
    loanDf['NPV'] = loanDf['Profit']/((1+discountRate/12)**loanDf['Period'])


    # roll it all up into a new aggregate dataframe
    loanTotal = loanDf.groupby(['MonthlyPayment'],
                            as_index=False).agg({'IntIncome': np.sum,
                                                 'TotalCost': np.sum,
                                                 'Profit': np.sum,
                                                 'NPV': np.sum})


    # set the columns to append last three: TotalCost, Profit, NPV
    cols = loanTotal.columns[2:].values.tolist()

    # create the colums in the original dataframe
    ploanPort = pd.concat([ploanPort,pd.DataFrame(columns=list(cols))])

    # set the columns to append last three: TotalCost, Profit, NPV
    cols = loanTotal.columns[2:].values.tolist()

    # create the colums in the original dataframe
    ploanPort = pd.concat([ploanPort,pd.DataFrame(columns=list(cols))])
    ploanPort.head()

    # put this in the porfolio DB
    j = 1

    for i in range(2,5):
        ploanPort.iloc[ln,j+5] = loanTotal.iloc[0,i]
        j += 1
    
    # go to the next loan
    ln +=1
    
    

# Get the Final Update Worksheet

In [12]:
ploanPort.head()

,ContractNum,ContractAmt,Term,APR,CalcPayment,Interest,TotalCost,Profit,NPV
0,3750024720,20000.0000,60.0000,0.0975,422.4849,5349.0924,2951.9357,2397.1567,2063.3638
1,3750024698,18000.0000,60.0000,0.0949,377.9455,4676.7327,2739.1191,1937.6136,1676.9526
2,3750024577,18000.0000,60.0000,0.0875,371.4702,4288.2113,2728.4095,1559.8018,1359.4844
3,3750024546,13000.0000,60.0000,0.1149,285.8386,4150.3182,2238.3255,1911.9927,1653.5599
4,3750024686,4500.0000,60.0000,0.1149,98.9441,1436.6486,1337.9888,98.6598,129.0712


In [13]:
# bind app id
ploanPort['AppId'] = df['AppId'].copy()
ploanPort.head()

,ContractNum,ContractAmt,Term,APR,CalcPayment,Interest,TotalCost,Profit,NPV,AppId
0,3750024720,20000.0000,60.0000,0.0975,422.4849,5349.0924,2951.9357,2397.1567,2063.3638,65398
1,3750024698,18000.0000,60.0000,0.0949,377.9455,4676.7327,2739.1191,1937.6136,1676.9526,65413
2,3750024577,18000.0000,60.0000,0.0875,371.4702,4288.2113,2728.4095,1559.8018,1359.4844,65423
3,3750024546,13000.0000,60.0000,0.1149,285.8386,4150.3182,2238.3255,1911.9927,1653.5599,65435
4,3750024686,4500.0000,60.0000,0.1149,98.9441,1436.6486,1337.9888,98.6598,129.0712,65441


In [14]:
finalPort = ploanPort[['AppId', 'ContractNum', 'ContractAmt','Term', 'APR', 'CalcPayment', 'Interest',
       'TotalCost', 'Profit', 'NPV']]
finalPort.head()

,AppId,ContractNum,ContractAmt,Term,APR,CalcPayment,Interest,TotalCost,Profit,NPV
0,65398,3750024720,20000.0000,60.0000,0.0975,422.4849,5349.0924,2951.9357,2397.1567,2063.3638
1,65413,3750024698,18000.0000,60.0000,0.0949,377.9455,4676.7327,2739.1191,1937.6136,1676.9526
2,65423,3750024577,18000.0000,60.0000,0.0875,371.4702,4288.2113,2728.4095,1559.8018,1359.4844
3,65435,3750024546,13000.0000,60.0000,0.1149,285.8386,4150.3182,2238.3255,1911.9927,1653.5599
4,65441,3750024686,4500.0000,60.0000,0.1149,98.9441,1436.6486,1337.9888,98.6598,129.0712


In [15]:
# write to csv
finalPort.to_csv('ploanPort_2021Q1Booked.csv', index=False)

In [16]:
finalPort.shape

(118, 10)